圖片來源:https://data.mendeley.com/datasets/7w7fccy7ky/4

In [ ]:
import os
import cv2
from ultralytics import YOLO
import csv

conf = 0.7
dir = "fall_detection.v4i.multiclass"
dir2 = "train"
# Path to the directory containing images
input_dir = f'C:/Users/User/Desktop/dataset/{dir}/{dir2}'  # Directory containing images
output_dir = f'C:/Users/User/Desktop/dataset/{dir}/{dir2}_detect_{conf}'  # Output directory for annotated images
output_file = f"C:/Users/User/Desktop/dataset/{dir}/{dir2}/{dir2}_keypoints_{conf}.csv"  # Output CSV file for keypoints data

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Load YOLOv8 model for pose detection
model = YOLO('yolov8n-pose.pt')  # Ensure you have the correct YOLOv8 pose model

def process_image(results):
    r = results[0]
    combined_results = []
    for i in range(len(r.keypoints.xyn)):
        for j in range(17):  # Assuming there are 17 keypoints
            x, y = r.keypoints.xyn[i][j].numpy()
            combined_results.append((x, y))
    box = results[0].boxes.xywhn.numpy().reshape(-1, 2)
    list_of_tuples = [tuple(row) for row in box]
    combined_results.extend(list_of_tuples)
    return combined_results

# Process each PNG image in the input directory
all_keypoints_data = []
# Get all files in input_dir
all_files = os.listdir(input_dir)
# Filter out only files that end with .png and are valid files
image_files = [filename for filename in all_files if os.path.isfile(os.path.join(input_dir, filename)) and filename.lower().endswith('.jpg')]
# Sort the filtered image files
image_files.sort()

for idx, filename in enumerate(image_files):
    input_path = os.path.join(input_dir, filename)
    
    # Output path remains the same as input path, with '_detect_confidence' appended
    output_path = os.path.join(output_dir, filename)

    # Load the image
    image = cv2.imread(input_path)
    
    # Perform pose detection
    results = model(image, conf=conf)

    if len(results[0].boxes) == 1:
        # Draw the results on the image
        annotated_image = results[0].plot()
        # Save the annotated image with the original filename
        cv2.imwrite(output_path, annotated_image)
        print(f"Processed and saved: {output_path}")
        keypoints_data = process_image(results)
        keypoints_data.insert(0, filename)  # Use the original filename
        all_keypoints_data.append(keypoints_data)
    else:
        keypoints_data = [filename] + [None] * 17  # Assuming 17 keypoints with x,y pairs
        all_keypoints_data.append(keypoints_data)

# Write keypoints data to CSV
with open(output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    
    # Write the header
    header = ["image_name"] + [f"x{i+1}, y{i+1}" for i in range(17)] + ["xb, yb"] + ["w, h"]  # Assuming 17 keypoints with x,y pairs
    writer.writerow(header)
    
    # Write the data
    writer.writerows(all_keypoints_data)

print("Data saved to", output_file)



0: 640x640 (no detections), 344.9ms
Speed: 5.0ms preprocess, 344.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 324.5ms
Speed: 4.1ms preprocess, 324.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Processed and saved: C:/Users/User/Desktop/dataset/fall_detection.v4i.multiclass/train_detect_0.7\132_png_jpg.rf.37b1886bc2a1d26b277f0f2bdd3dd50f.jpg

0: 640x640 2 persons, 322.1ms
Speed: 4.0ms preprocess, 322.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 319.2ms
Speed: 4.0ms preprocess, 319.2ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
Processed and saved: C:/Users/User/Desktop/dataset/fall_detection.v4i.multiclass/train_detect_0.7\134_png_jpg.rf.3ee2c1420fc279d85880613d90d6f53c.jpg

0: 640x640 1 person, 340.9ms
Speed: 4.0ms preprocess, 340.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Processed and saved: C:/Users/User/Desktop/dataset/f

In [21]:
import os
import pandas as pd
s = 2
# Specify the directory containing the CSV files
directory = f'C:/Users/User/Desktop/Dataset CAUCAFall/CAUCAFall/Subject.{s}'

# List all files in the directory
files = os.listdir(directory)

# Filter the list to include only CSV files
csv_files = [file for file in files if file.endswith('.csv')]

# Initialize an empty list to store DataFrames
dataframes = []

# Read each CSV file into a DataFrame and append it to the list
for file in csv_files:
    file_path = os.path.join(directory, file)
    df = pd.read_csv(file_path)
    dataframes.append(df)

# Concatenate all DataFrames along axis 0
combined_df = pd.concat(dataframes, axis=0)

# Reset the index of the combined DataFrame
combined_df.reset_index(drop=True, inplace=True)

# Display the combined DataFrame
print(combined_df)

# Specify the path for the output CSV file
output_file = f'C:/Users/User/Desktop/Dataset CAUCAFall/CAUCAFall/combined_Subject_{s}.csv'

# Export the combined DataFrame to a CSV file
combined_df.to_csv(output_file, index=False)


            image_name                    x1, y1                    x2, y2  \
0    detected_0000.png                (0.0, 0.0)  (0.22899742, 0.33749762)   
1    detected_0001.png  (0.23111638, 0.33650577)   (0.22894971, 0.3256766)   
2    detected_0002.png   (0.22828984, 0.3290265)   (0.2275349, 0.31944948)   
3    detected_0003.png   (0.23104408, 0.3312378)   (0.22913784, 0.3204318)   
4    detected_0004.png  (0.23130834, 0.33479673)   (0.22917111, 0.3241199)   
..                 ...                       ...                       ...   
638  detected_0153.png   (0.5969123, 0.57490796)  (0.59923273, 0.56798416)   
639  detected_0154.png     (0.59510493, 0.56869)                (0.0, 0.0)   
640  detected_0155.png  (0.59754044, 0.57325363)    (0.59906614, 0.565999)   
641  detected_0156.png   (0.59725213, 0.5720937)  (0.59587914, 0.56271356)   
642  detected_0157.png    (0.5970473, 0.5742124)   (0.59897286, 0.5671038)   

                      x3, y3                    x4, y4  \
0    